In [71]:
import boto3

BUCKET_NAME = 'mini-project-bucket-text'

def download_file(BUCKET_NAME):

    objects = []
    s3 = boto3.client('s3', aws_access_key_id = " ", 
                    aws_secret_access_key = " ")
    
    for key in s3.list_objects(Bucket=BUCKET_NAME)['Contents']:
        objects.append(key['Key'])
    
    BUCKET_FILE_NAME = objects[-1]
    s3.download_file(BUCKET_NAME, BUCKET_FILE_NAME, BUCKET_FILE_NAME)
    return BUCKET_FILE_NAME



In [72]:
LOCAL_FILE_NAME = download_file(BUCKET_NAME)

In [73]:
import PyPDF2

def get_file_contents(LOCAL_FILE_NAME):
   pdf_object = open(LOCAL_FILE_NAME, 'rb')
   pdf_reader = PyPDF2.PdfFileReader(pdf_object)
   num_pages = pdf_reader.numPages
   text = []
   for i in range(num_pages):
      page_object = pdf_reader.getPage(i)
      text += page_object.extract_text()
   
   pdf_object.close()
   original_text = ''.join(text)
   formatted_text = original_text.replace('\\n', '\n')
   return formatted_text

In [74]:
text = get_file_contents(LOCAL_FILE_NAME)
# text = """Federer won the Swiss Indoors last week by beating Romanian qualifier Marius Copil in
# the final. The 37-year-old claimed his 99th ATP title and is hunting the century in
# the French capital this week. Federer has been handed a difficult draw where could
# could come across Kevin Anderson, Novak Djokovic and Rafael Nadal in the latter
# rounds. But first the 20-time Grand Slam winner wants to train on the Paris Masters
# court this afternoon before deciding whether to appear for his opening match against
# either Milos Raonic or Jo-Wilfried Tsonga. "On Monday, I am free and will look how I
# feel," Federer said after winning the Swiss Indoors. "On Tuesday I will fly to Paris
# and train in the afternoon to be ready for my first match on Wednesday night. "I felt
# good all week and better every day. "We also had the impression that at this stage it
# might be better to play matches than to train. "And as long as I fear no injury, I
# play." Federer's success in Basel last week was the ninth time he has won his
# hometown tournament. And he was delighted to be watched on by all of his family and
# friends as he purchased 60 tickets for the final for those dearest to him. "My
# children, my parents, my sister and my team are all there," Federer added. "It is
# always very emotional for me to thank my team. And sometimes it tilts with the
# emotions, sometimes I just stumble. "It means the world to me. It makes me incredibly
# happy to win my home tournament and make people happy here. "I do not know if it's
# maybe my last title, so today I try a lot more to absorb that and enjoy the moments
# much more consciously. "Maybe I should celebrate as if it were my last title. "There
# are very touching moments: seeing the ball children, the standing ovations, all the
# familiar faces in the audience. Because it was not always easy in the last weeks."""
# with open('readme.txt', 'w', encoding='utf-8') as f:
#     f.write(text)
len(text)

Xref table not zero-indexed. ID numbers for objects will be corrected.


24593

In [75]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation

stopwords = STOP_WORDS

nlp = spacy.load('en_core_web_sm')

doc = nlp(text)

tokens = [token.text for token in doc]


In [76]:
punctuation = punctuation + '\n'

word_frequencies={}
for word in doc :
    if word.text.lower() not in stopwords:
        if word.text.lower() not in punctuation:
            if word.text not in word_frequencies.keys():
                word_frequencies[word.text] = 1
            else:
                word_frequencies[word.text] += 1

# print(word_frequencies)

In [77]:
max_frequency = max(word_frequencies.values())
max_frequency


60

In [78]:
for word in word_frequencies.keys():
    word_frequencies[word]= word_frequencies[word]/max_frequency

# print(word_frequencies)

In [79]:
sentence_tokens = [sent for sent in doc.sents]
# print(sentence_tokens)

In [80]:
sentence_score = {}
for sent in sentence_tokens:
    for word in sent:
        if word.text.lower() in word_frequencies.keys():
            if sent not in sentence_score.keys():
                sentence_score[sent]=word_frequencies[word.text.lower()]
            else:
                sentence_score[sent]+=word_frequencies[word.text.lower()]

# sentence_score

In [81]:
from  heapq import nlargest

select_length = int(len(sentence_tokens)*0.3)
select_length

summary = nlargest(select_length,sentence_score,key=sentence_score.get)
# summary

In [82]:
summary = nlargest(select_length,sentence_score,key=sentence_score.get)
# summary

In [83]:
final_summary =[word.text for word in summary]
# print(final_summary)

In [84]:
summary = " ".join(final_summary)
# print(summary)

In [85]:
len(summary)

14838

In [86]:
with open('test.txt', 'w', encoding='utf-8') as f:
    f.write(summary)
f.close()

In [87]:
import textwrap
from fpdf import FPDF

def text_to_pdf(text, filename):
    a4_width_mm = 210
    pt_to_mm = 0.35
    fontsize_pt = 10
    fontsize_mm = fontsize_pt * pt_to_mm
    margin_bottom_mm = 10
    character_width_mm = 7 * pt_to_mm
    width_text = a4_width_mm / character_width_mm

    pdf = FPDF(orientation='P', unit='mm', format='A4')
    pdf.set_auto_page_break(True, margin=margin_bottom_mm)
    pdf.add_page()
    pdf.set_font(family='Courier', size=fontsize_pt)
    splitted = text.split('\n')

    for line in splitted:
        lines = textwrap.wrap(line, width_text)

        if len(lines) == 0:
            pdf.ln()

        for wrap in lines:
            pdf.cell(0, fontsize_mm, wrap, ln=1)

    pdf.output(filename, 'F')

In [88]:
input_filename = 'test.txt'
output_filename = 'summarized.pdf'
file = open(input_filename, encoding='latin-1')
text = file.read()
file.close()
text_to_pdf(text, output_filename)